In [1]:
from pylearn2.datasets.mnist import MNIST
from pylearn2.train import Train
from pylearn2.models import softmax_regression, mlp
from pylearn2.training_algorithms import bgd, sgd
from pylearn2.termination_criteria import MonitorBased
from pylearn2.train_extensions import best_params
from pylearn2.utils import serial
from pylearn2.costs.mlp.dropout import Dropout
from theano import function
from theano import tensor as T
import numpy as np
import os
from __future__ import division
from time import time

In [2]:
train = MNIST('train', start=0, stop=50000)

In [3]:
print "training space : %s" %str(train.X_space)
print "training space : %s rows  %s columns" %(train.X.shape[0], train.X.shape[1])
print "unique members of lable : %s " %str(np.unique(train.y))

training space : VectorSpace(dim=784, dtype=float64)
training space : 50000 rows  784 columns
unique members of lable : [0 1 2 3 4 5 6 7 8 9] 


In [4]:
h0 = mlp.Softplus(layer_name='h0', dim=800, sparse_init=40)
h1 = mlp.Softplus(layer_name='h1', dim=800, sparse_init=40)
# h2 = mlp.Softplus(layer_name='h2', dim=50, sparse_init=15)
ylayer = mlp.Softmax(layer_name='y', n_classes=10, irange=0)
layers = [h0, h1, ylayer]

In [5]:
model = mlp.MLP(layers, nvis=784)
train = MNIST('train', start=0, stop=50000)
valid = MNIST('train', start=50000, stop=60000)
test = MNIST('test', start=0, stop=10000)

In [6]:
monitoring = dict(valid=valid)
termination = MonitorBased(channel_name="valid_y_misclass", N=100)
extensions = [best_params.MonitorBasedSaveBest(channel_name="valid_y_misclass",
save_path="train_best.pkl")]

In [ ]:
algorithm = sgd.SGD(0.1, batch_size=100, cost=Dropout(),
                    monitoring_dataset = monitoring, termination_criterion = termination)

In [ ]:
save_path = "train_best.pkl"
t0 = time()
if os.path.exists(save_path):
    model = serial.load(save_path)
else:
    print 'Running training'
    train_job = Train(train, model, algorithm, extensions=extensions, save_path="train.pkl", save_freq=1)
    train_job.main_loop()
print "escape time : ", round(time()-t0, 3), "s"

Running training
Parameter and initial learning rate summary:


/Users/dikien/anaconda/lib/python2.7/site-packages/pylearn2/train.py:85: UserWarning: dataset has no yaml src, model won't know what data it was trained on
  "data it was trained on")


	h0_W: 0.1
	h0_b: 0.1
	h1_W: 0.1
	h1_b: 0.1
	softmax_b: 0.1
	softmax_W: 0.1
Compiling sgd_update...
Compiling sgd_update done. Time elapsed: 1.116040 seconds
compiling begin_record_entry...
compiling begin_record_entry done. Time elapsed: 0.150263 seconds
Monitored channels: 
	learning_rate
	total_seconds_last_epoch
	training_seconds_this_epoch
	valid_h0_col_norms_max
	valid_h0_col_norms_mean
	valid_h0_col_norms_min
	valid_h0_max_x_max_u
	valid_h0_max_x_mean_u
	valid_h0_max_x_min_u
	valid_h0_mean_x_max_u
	valid_h0_mean_x_mean_u
	valid_h0_mean_x_min_u
	valid_h0_min_x_max_u
	valid_h0_min_x_mean_u
	valid_h0_min_x_min_u
	valid_h0_range_x_max_u
	valid_h0_range_x_mean_u
	valid_h0_range_x_min_u
	valid_h0_row_norms_max
	valid_h0_row_norms_mean
	valid_h0_row_norms_min
	valid_h1_col_norms_max
	valid_h1_col_norms_mean
	valid_h1_col_norms_min
	valid_h1_max_x_max_u
	valid_h1_max_x_mean_u
	valid_h1_max_x_min_u
	valid_h1_mean_x_max_u
	valid_h1_mean_x_mean_u
	valid_h1_mean_x_min_u
	valid_h1_min_x_max_

/Users/dikien/anaconda/lib/python2.7/site-packages/pylearn2/monitor.py:572: UserWarning: Trained model saved without indicating yaml_src
  'indicating yaml_src')


Time this epoch: 17.459340 seconds
Monitoring step:
	Epochs seen: 1
	Batches seen: 500
	Examples seen: 50000
	learning_rate: 0.1
	total_seconds_last_epoch: 0.0
	training_seconds_this_epoch: 17.45934
	valid_h0_col_norms_max: 8.95849760275
	valid_h0_col_norms_mean: 6.31649684628
	valid_h0_col_norms_min: 4.02562489058
	valid_h0_max_x_max_u: 24.2454379314
	valid_h0_max_x_mean_u: 2.6806868412
	valid_h0_max_x_min_u: 0.0639223198311
	valid_h0_mean_x_max_u: 13.7350825503
	valid_h0_mean_x_mean_u: 0.771428219771
	valid_h0_mean_x_min_u: 0.00339624414824
	valid_h0_min_x_max_u: 6.21969609993
	valid_h0_min_x_mean_u: 0.132779166783
	valid_h0_min_x_min_u: 9.56576699942e-08
	valid_h0_range_x_max_u: 19.2563496543
	valid_h0_range_x_mean_u: 2.54790767442
	valid_h0_range_x_min_u: 0.0639189762871
	valid_h0_row_norms_max: 9.150629016
	valid_h0_row_norms_mean: 6.36265596668
	valid_h0_row_norms_min: 3.71222707762
	valid_h1_col_norms_max: 8.74417111975
	valid_h1_col_norms_mean: 6.27747148424
	valid_h1_col_norms

In [ ]:
X = model.get_input_space().make_batch_theano()
Y = model.fprop(X)